In [1]:
import pyspark as ps
from bs4 import BeautifulSoup as bs
import json

spark = (ps.sql.SparkSession.builder
        .appName("sandbox")
        .getOrCreate()
        )

sc = spark.sparkContext

In [2]:
def import_records(json_file):
    records = []
    with open(json_file) as f:
        for i in f:
            records.append(json.loads(i))
    return records

def separate_json(json_file):
    records = import_records(json_file)
    id_lst = []
    url_lst = []
    soup_lst = []
    time_lst = []
    for i in range(len(records)):
        id_lst.append(records[i]['_id'])
        url_lst.append(records[i]['url'])
        soup_lst.append(bs(records[i]['html'], 'html.parser'))
        time_lst.append(records[i]['time'])
    return id_lst, url_lst, soup_lst, time_lst

def pipeline(json_file):
    id_lst, url_lst, soup_lst, time_lst = separate_json(json_file)
    content = []
    other_stuff = []
    for i in range(len(soup_lst)):
        td_tags = soup_lst[i].find_all('td')
        other_stuff.append(td_tags[0].text)
        content.append(td_tags[1].text)
    contents_lower = []
    for val in content:
        if val == None:
            contents_lower.append("")
        else:
            contents_lower.append(val.lower())
    words_to_remove = ['Occurred', 'Reported', 'Posted', 
                       'Location', 'Shape', 'Duration']
    features_dic = {'Occurred': [],'Location': [],
                    'Shape': [], 'Duration': []}
    
    for i in range(len(other_stuff)):
        removekeys = other_stuff[i]
        for val in words_to_remove:
            removekeys = removekeys.replace(val,"")
        lst = removekeys.split(':')
        try:
            features_dic['Occurred'].append(lst[1])
        except:
            features_dic['Occurred'].append(None)
        try:
            features_dic['Location'].append(lst[10])
        except:
            features_dic['Location'].append(None)
        try:
            features_dic['Shape'].append(lst[11])
        except:
            features_dic['Shape'].append(None)
        try:
            features_dic['Duration'].append(lst[12])
        except:
            features_dic['Duration'].append(None)
    return features_dic, contents_lower

In [ ]:
file = '/home/jovyan/work/Galvanize/DSI/repos/nlp/ufodata.json'
dic, content = pipeline(file)

'/home/jovyan'